# Initialization

In [1]:
import matplotlib.pyplot as plt
import xarray as xr
import numpy as np

plt.rcParams['figure.figsize'] = [16, 3]

import sys
sys.path.append('/mnt/lustre/IAM851/jm1667/psc/python')
import psc

import logging

%config InlineBackend.figure_format = 'retina'

In [115]:
ks = [3, 5, 15]
As = [1, 5, 10, 20]
phi0s = [0, 0, 0, 0]

xmin, xmax = 0, 2*np.pi
dx = (xmax - xmin) / 32
ndims = 1
dimnames = ['x', 'y', 'z']

tmin = 0
dt = dx * 1.1
tmax = 128 * dt

c = .5

def getStep(time: float, every: int = 1):
    return int(time // dt) * every

# Animation
from https://stackoverflow.com/questions/18743673/show-consecutive-images-arrays-with-imshow-as-repeating-animation-in-python
and http://louistiao.me/posts/notebooks/embedding-matplotlib-animations-in-jupyter-notebooks/

In [106]:
import matplotlib.animation as animation
from IPython.display import HTML
import numpy as np

xss = [np.arange(xmin, xmax, dx) for n in range(ndims)]

Xss = np.meshgrid(*xss)

In [107]:
def addSound(da, t: float, k: int, A: float, phi0: float):
    da.values += A * np.sin(sum(k * Xs for Xs in Xss) + k*c*t + phi0)

def getDaSlice(t: float):
    da = xr.DataArray(np.zeros_like(Xss[0]), coords=dict(zip(dimnames, xss)), dims=dimnames[:ndims])
    for k, A, phi0 in zip(ks, As, phi0s):
        addSound(da, t, k, A, phi0)
    return da

In [116]:
# construct data
ts = np.arange(tmin, tmax, dt)
da = xr.concat([getDaSlice(t) for t in ts], 't').assign_coords(t=("t", ts))

In [117]:
# view tmin
%matplotlib ipympl
fig, ax = plt.subplots()
im = da.sel(t=0).plot(ax=ax)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [70]:
# make movie
%matplotlib ipympl
def updateIm(frameIdx):
    nextData = da.isel(t=frameIdx)
    if isinstance(im, list):
        im[0].set_data(xss[0], nextData)
    else:
        im.set_array(nextData)
    return [im]


anim = animation.FuncAnimation(fig, updateIm, interval=30, frames=len(ts), repeat=False, blit=True)
HTML(anim.to_html5_video())

## Stability Plot

In [118]:
%matplotlib ipympl
def norm(x):
    return xr.apply_ufunc(np.linalg.norm, x, input_core_dims=[dimnames[:ndims]])

normsOfDiffs = [norm(da.isel(t=it) - da.isel(t=0)) for it in range(len(ts))]

plt.xlabel("Time")
plt.ylabel("2-Norm of Difference")
plt.title("Stability")

plt.plot(ts, normsOfDiffs)
plt.show()
periods = [2*np.pi/(c*abs(k)) for k in ks]
print("periods should be " + ', '.join("{:.2f}".format(T) for T in periods))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

periods should be 4.19, 2.51, 0.84


In [119]:
def lcm(vals, tolerance=.01):
    maybeLCMs = np.array(vals)
    while not np.allclose(maybeLCMs[1:], maybeLCMs[:-1]):
        minidx = np.argmin(maybeLCMs)
        maybeLCMs[minidx] += vals[minidx]
    return maybeLCMs.mean()

period = lcm(periods)
print("Overall period is {:.2f}".format(period))

Overall period is 12.57


# FFT

In [120]:
import xrft
import matplotlib

In [124]:
T = period#s[2]

Fda = da#.sel(t=slice(0,(tmax // T) * T))
for dim in da.dims:
    print(f"DFTing {dim}...")
    Fda = xrft.dft(Fda, dim=dim, true_phase=True, true_amplitude=True)
print("Done.")

Fda2d = Fda
if 'freq_y' in Fda2d.dims:
    Fda2d = Fda2d.isel(freq_y=0)
if 'freq_z' in Fda2d.dims:
    Fda2d = Fda2d.isel(freq_z=0)

DFTing t...
DFTing x...
Done.


In [103]:
IFda = Fda
for dim in Fda.dims:
    print(f"IDFTing {dim}...")
    IFda = xrft.idft(IFda, dim=dim, true_phase=True, true_amplitude=True)
print("Done.")

IFda

IDFTing freq_t...
IDFTing freq_x...
Done.


<xarray.DataArray (t: 128, x: 32)>
array([[ 8.14947204e-14-1.09112408e-15j, -2.66638215e+01+1.11182588e-15j,
        -1.71644287e+00+2.88890110e-16j, ...,
        -1.24880608e+01-2.71867842e-15j,  1.71644287e+00+1.37731920e-15j,
         2.66638215e+01-2.13327033e-16j],
       [ 1.54302000e+00-2.53227107e-16j,  1.79626049e+01-6.25993333e-16j,
        -2.71284994e+01-1.50038184e-16j, ...,
         2.71284994e+01-3.06408116e-15j, -1.79626049e+01+7.86993959e-16j,
        -1.54302000e+00-2.47960505e-16j],
       [-2.66638215e+01+1.26096651e-15j, -1.71644287e+00-9.56651200e-16j,
         1.24880608e+01-3.14162533e-16j, ...,
         1.71644287e+00-8.36761559e-16j,  2.66638215e+01+1.37409838e-15j,
        -2.39056330e-13+1.71323822e-15j],
       ...,
       [-1.79626049e+01+3.58228337e-15j, -1.54302000e+00-2.50420554e-16j,
         1.54302000e+00-1.12471055e-15j, ...,
        -1.21115077e+01-1.22955888e-16j,  5.84252897e+00-1.62872644e-15j,
         2.71284994e+01-2.58009064e-15j],
       [ 2.66638215e+01+2.37765947e-15j, -2.03802253e-13+2.37142251e-15j,
        -2.66638215e+01+7.23700610e-16j, ...,
        -9.89949494e+00+1.44554644e-15j, -1.24880608e+01+4.50682449e-16j,
         1.71644287e+00+1.62923040e-15j],
       [-1.54302000e+00-5.82651733e-16j,  1.54302000e+00+1.75251196e-16j,
         1.79626049e+01-1.34279422e-15j, ...,
         5.84252897e+00+9.66304198e-16j,  2.71284994e+01+5.03737078e-16j,
        -1.79626049e+01+6.71328754e-16j]])
Coordinates:
  * t        (t) float64 -12.57 -12.37 -12.17 -11.98 ... 11.78 11.98 12.17 12.37
  * x        (x) float64 -3.142 -2.945 -2.749 -2.553 ... 2.356 2.553 2.749 2.945

In [125]:
%matplotlib ipympl
fig, (ax1, ax2) = plt.subplots(figsize=(12,4), nrows=1, ncols=2)

Fda2d.real.plot(ax=ax1, vmax=np.nanquantile(Fda2d.real.data, 1))
ax1.set_title("Real Part")
Fda2d.imag.plot(ax=ax2, vmax=np.nanquantile(Fda2d.imag.data, 1))
ax2.set_title("Imaginary Part")
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [145]:
%matplotlib ipympl
fig, ax = plt.subplots()

pltme = (Fda2d.real **2 + Fda2d.imag**2).sel(freq_t=slice(0,None), freq_x=slice(0,None)) ** .5
dimswapper = {dim:dim.replace('freq_', 'k') for dim in pltme.dims} | {'freq_t':'om'}
pltme = pltme.assign_coords({dimswapper[dim]: pltme.coords[dim] * 2 * np.pi for dim in pltme.dims}).swap_dims(dimswapper)

pltme.real.plot(ax=ax)
ax.set_title("Magnitude")
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Numpy FFT

In [85]:
_Fda = np.fft.rfft2(da.sel(t=slice(0,(tmax // T) * T)).data)
Fda = xr.DataArray(_Fda, )

In [92]:
%matplotlib ipympl
fig, ax = plt.subplots()

# pltme = (Fda2d.real **2 + Fda2d.imag**2).sel(t=slice(0,None), freq_x=slice(0,None)) ** .5


plt.pcolormesh(Fda.imag)
# Fda.plot(ax=ax, vmax=np.nanquantile(pltme.real.data, 0.999))
ax.set_title("Magnitude")
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Radial Dependence

In [99]:
import matplotlib

kgrid = (Fda.freq_x ** 2 + Fda.freq_y**2) ** .5
maxK = kgrid.max()
kstep = maxK / 256

def getMean(data, k):
    kslice = data.where((k <= kgrid) & (kgrid < k + kstep))
    return kslice.mean().item()

ks = np.arange(0, maxK, kstep)
def getOmK(omKyKz):
    return xr.DataArray([[getMean(omKyKz[om,:,:], k) for k in ks] for om in range(len(omKyKz.freq_t))], dims=['ω', 'k'], coords=[omKyKz.freq_t.data, ks])

In [100]:
omK = getOmK(Fda)
omK

<xarray.DataArray (ω: 425, k: 256)>
array([[ 1.52143935e-12-1.32761946e-12j,  1.56194347e-12-1.29371991e-12j,
         1.81333543e-12-1.57833696e-12j, ...,
         1.98088611e-12-4.61408185e-12j, -1.68687585e-11+1.24174513e-11j,
        -1.99010216e-11+1.38604901e-11j],
       [-1.06102951e-12-1.70870143e-12j, -1.06822202e-12-1.74521854e-12j,
        -1.27076267e-12-2.06132743e-12j, ...,
         2.24502975e-12-2.94142451e-12j,  1.59010832e-11+1.84909666e-11j,
         1.71166715e-11+2.22669934e-11j],
       [-1.89635031e-12+8.00733369e-13j, -1.88649621e-12+7.81171204e-13j,
        -2.27491993e-12+9.11532394e-13j, ...,
        -9.18786472e-12+1.30677188e-12j,  1.33228428e-11-7.07691763e-12j,
         1.56904182e-11-7.81399598e-12j],
       ...,
       [-1.89635031e-12-8.00733369e-13j, -1.88649621e-12-7.81171204e-13j,
        -2.27491993e-12-9.11532394e-13j, ...,
        -9.18786472e-12-1.30677188e-12j,  1.33228428e-11+7.07691763e-12j,
         1.56904182e-11+7.81399598e-12j],
       [-1.06102951e-12+1.70870143e-12j, -1.06822202e-12+1.74521854e-12j,
        -1.27076267e-12+2.06132743e-12j, ...,
         2.24502975e-12+2.94142451e-12j,  1.59010832e-11-1.84909666e-11j,
         1.71166715e-11-2.22669934e-11j],
       [ 1.52143935e-12+1.32761946e-12j,  1.56194347e-12+1.29371991e-12j,
         1.81333543e-12+1.57833696e-12j, ...,
         1.98088611e-12+4.61408185e-12j, -1.68687585e-11-1.24174513e-11j,
        -1.99010216e-11-1.38604901e-11j]])
Coordinates:
  * ω        (ω) float64 -48.16 -47.93 -47.7 -47.48 ... 47.48 47.7 47.93 48.16
  * k        (k) float64 0.0 70.71 141.4 212.1 ... 1.789e+04 1.796e+04 1.803e+04

In [101]:
%matplotlib ipympl
fig, ax = plt.subplots()

pltme = (omK.real **2 + omK.imag**2).sel(indexers={'ω':slice(0,None)}) ** .5

pltme.real.plot(ax=ax, vmax=np.nanquantile(pltme.real.data, .99), norm=matplotlib.colors.LogNorm())
ax.set_title("Magnitude")
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …